In [1]:
#imports
import numpy as np
import math

In [2]:
# Load Data
data = np.genfromtxt('scale-d-10d.csv', delimiter=' ')
data = data[:,0:9]

labels = np.genfromtxt('scale-d-10d.csv', delimiter=' ',dtype="|U5")
labels = labels[:,10]

In [4]:
def proj_true_p_signature(M):
    
    # For every bin in every attribute, its support is computed
    supp_sets = compute_support(M)
    


    return None

In [ ]:
# Compute Support set of all attributes
# SupportSet = {x ∈ D | x.aj ∈ S }
def compute_support(M):
    # n = number of data objects
    n = M.shape[0]
    # number of attributes 
    attribute_number = M.shape[1]
    
    bin_number = int(1 + math.log(n,2))
    list_of_supports = []
    
    supp_set = [[]] * bin_number
    return list_of_supports

In [5]:
print(data)
print(data.shape)

print(labels)
print(labels.shape)


print(int(1 + math.log(data.shape[0],2)))

[[ 0.65264191 -0.00371625  0.01521223 ...  0.21708319  0.78426444
   0.27407853]
 [ 1.01930291  0.35519293  0.47796839 ...  0.60008908  0.35612478
   0.24724175]
 [ 0.5940366  -0.33190342 -0.10154468 ...  0.68686449  0.53546747
   1.27822433]
 ...
 [ 0.84569877  0.35925799  0.64490759 ...  0.87088832  0.63797899
   0.87146842]
 [ 0.27499317  0.49355767  0.20943054 ...  0.93080848  0.72043327
   1.42526599]
 [ 0.39640662  0.57944655  0.980204   ...  0.0876678   0.15534795
   0.67756318]]
(10000, 9)
['C1' 'C1' 'C1' ... 'C100' 'C100' 'C100']
(10000,)
14


In [163]:
# Extra
M = np.array([[1, 2], [4, 5], [3, 3], [6, 7]])
n = M.shape[0]
attribute_number = M.shape[1]
bin_number = int(1 + math.log(n,2))
list_of_supports = []

for i in range(attribute_number):
    supp_set = [[]] * bin_number
    
    for j in range(n):
        if M[j,i] < 3:
            
        
        
    
    list_of_supports.append(supp_set)



list_of_supports

[[[array([1, 2])], [array([1, 2])], [array([1, 2])]],
 [[array([1, 2])], [array([1, 2])], [array([1, 2])]]]

In [170]:
supp_set = [[]] * 3
supp_set[0] = [M[1], M[0]]

supp_set

[[array([4, 5]), array([1, 2])], [], []]

[1, 2, 3, 4]